<a target="_blank" href="https://colab.research.google.com/github/trainocate-japan/Machine-Learning-and-Deep-Learning-Hands-on/blob/main/answer/2_線形回帰/2-1-1_線形回帰による中古マンションの価格予測①.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 2-1-1_線形回帰による中古マンションの価格予測

このノートブックでは、機械学習の基本的な作業工程や機械学習でよく使うライブラリを試しながら、線形回帰で予測モデルを作成します。

予測を行うテーマは物件の価格を予測することです。<br>
神奈川県にある、JR東海道線の戸塚駅周辺物件情報を用意しました。2019年4月～2021年3月までの2年間で売買が成立した中古マンションの、住宅価格と価格以外の情報を確認して、価格以外の情報から住宅価格を予測してみましょう。

## ライブラリのインポート

In [1]:
# データを処理するための基本的なライブラリ
# どれも途中で使用することが多いので、使うかどうかにかかわらずひとまずインポートしてしまってもよいでしょう
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# scikit-learnから機械学習モデルをインポート。今回は線形回帰モデル。
from sklearn.linear_model import LinearRegression

In [3]:
'''
matplotlibに日本語を表示させたいときには以下のコマンドを実行して、japanize_matplotlibをインストールしておく。
Google Colaboratoryにプリインストールされていないので、インストール作業が必要。
'''
# !を先頭に入れるとBashコマンドを入力できる。japanize_matplotlibをインストールする。
!pip install japanize_matplotlib | tail -n 1
# japanize_matplotlibをインポート
import japanize_matplotlib

## データの準備

#### データを取り込む
- pandasのread_csvメソッドを使用して、mlho/data/totsuka_used.csvファイルを読み込みます
- read_csvメソッドは引数に指定したファイルをpandasのDataFrame型で返します

In [ ]:
# csvファイルを読み込みます
totsuka_used = pd.read_csv("totsuka_used.csv")

#### データを確認する

In [6]:
# 読み込んだデータを確認します
# DataFrameに用意されているheadメソッドは先頭の5行分のデータを出力します。
totsuka_used.head()

,地区名,最寄駅距離（分）,取引価格,間取り,面積（㎡）,建築年,築年数,構造,都市計画,建ぺい率（％）,容積率（％）,取引年,改装
0,秋葉町,45,35000000,３ＬＤＫ,65,平成29年,4,ＲＣ,工業地域,60,200,2021,未改装
1,秋葉町,45,19000000,３ＬＤＫ,60,昭和54年,42,ＲＣ,準工業地域,60,200,2021,改装済
2,秋葉町,45,36000000,３ＬＤＫ,65,平成29年,4,ＲＣ,工業地域,60,200,2021,未改装
3,秋葉町,45,6000000,３ＤＫ,60,昭和54年,41,ＲＣ,準工業地域,60,200,2020,未改装
4,秋葉町,45,32000000,３ＬＤＫ,65,平成29年,3,ＲＣ,工業地域,60,200,2020,未改装


データの全体像を確認します。

取り込んだデータが意図したとおりの型・形状になっているか、欠損値（歯抜けになっている値）がないか、最小・最大・平均値の値はどうなっているかなど、機械学習を行う前にこれから予測するデータについて理解を深めておきます。

In [7]:
# totsuka_usedのデータ要約を確認
totsuka_used.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   地区名       238 non-null    object
 1   最寄駅距離（分）  238 non-null    int64 
 2   取引価格      238 non-null    int64 
 3   間取り       238 non-null    object
 4   面積（㎡）     238 non-null    int64 
 5   建築年       238 non-null    object
 6   築年数       238 non-null    int64 
 7   構造        238 non-null    object
 8   都市計画      238 non-null    object
 9   建ぺい率（％）   238 non-null    int64 
 10  容積率（％）    238 non-null    int64 
 11  取引年       238 non-null    int64 
 12  改装        238 non-null    object
dtypes: int64(7), object(6)
memory usage: 24.3+ KB


In [8]:
# totsuka_usedの統計情報を確認
totsuka_used.describe()

,最寄駅距離（分）,取引価格,面積（㎡）,築年数,建ぺい率（％）,容積率（％）,取引年
count,238.00000,238.00000,238.00000,238.00000,238.00000,238.00000,238.00000
mean,27.34874,"24,381,512.60504",68.34034,25.89916,60.50420,195.12605,"2,019.68067"
std,17.10455,"14,087,615.93020",13.94223,14.33970,4.65678,69.88677,0.62888
min,1.00000,"1,500,000.00000",15.00000,2.00000,40.00000,80.00000,"2,019.00000"
25%,14.00000,"14,000,000.00000",60.00000,14.00000,60.00000,150.00000,"2,019.00000"
50%,24.00000,"24,000,000.00000",70.00000,24.50000,60.00000,200.00000,"2,020.00000"
75%,45.00000,"32,750,000.00000",75.00000,36.00000,60.00000,200.00000,"2,020.00000"
max,75.00000,"70,000,000.00000",110.00000,53.00000,80.00000,600.00000,"2,021.00000"


#### 説明変数、目的変数を切り出す
- 説明変数はtotsuka_usedから「築年数」を選択し、変数名をtrain_xとして代入しましょう
- 目的変数はtotsuka_usedから「取引価格」を選択し、変数名をtrain_yとして代入しましょう

In [9]:
# 列の指定に[]を二重に記載していますが、これは2次元の形式（DataFrame型）であることを明示するためです
train_x = totsuka_used[['築年数']]

In [10]:
train_y = totsuka_used['取引価格']

## 機械学習モデルを定義する


In [11]:
# 変数modelに線形回帰の機械学習モデルのインスタンスを格納する
model = LinearRegression()

## 機械学習モデルの学習
- 作成したモデルのインスタンスに対してfitメソッドを使います
- 引数として、用意した、説明変数、目的変数を順に指定します
- fitを実行すると与えたデータを用いて学習を行い、予測を行うことができるようになります
- データをNumPy配列ではなくDataFrameで渡していますが、自動で変換が行われます。

In [12]:
model.fit(train_x, train_y)

LinearRegression()

## 機械学習モデルの評価
出来上がった線形回帰の方程式を確認してみます。

In [13]:
# 傾きを求めるにはcoef_プロパティを使う
# y = ax + bのaの部分
model.coef_

array([-725666.857])

In [14]:
# 切片を求めるにはintercept_プロパティを使う
# y = ax + bのbの部分
model.intercept_

43175674.392

出来上がった方程式は「y = -725666x + 43175674」です。<br>
築年数が0年の時4300万円と予測し、築年数が一年増えるごとに70万円ほど価格が減少するような計算式となりました。

#### 予測をする
- 予測をするにはpredictメソッドを使います
- predictメソッドの引数に説明変数を指定します
- predictメソッドの引数はnumpy配列です。複数まとめてデータを渡すことができます。一件だけ予測したい場合にも配列で渡します。（ここでもDataFrameが自動で変換されます）
- 同じくpredictメソッドの戻り値はnumpy配列です。複数まとめて結果を受け取ることができます。

In [15]:
# train_xの値をすべて渡して予測し、そのうち最初の5件だけを表示する
model.predict(train_x)[:5]

array([40273006.965, 12697666.406, 40273006.965, 13423333.263,
       40998673.822])

In [16]:
# 正解を確認してみる
train_y.head()

0    35000000
1    19000000
2    36000000
3     6000000
4    32000000
Name: 取引価格, dtype: int64

In [17]:
# 入力されたtrain_xの値も確認してみる
train_x.head(5)

,築年数
0,4
1,42
2,4
3,41
4,3


#### 精度を確認する
評価は線形回帰のモデルに用意されているscoreメソッドで行います。<br>
回帰の場合の精度は決定係数R2で計算されます。

In [18]:
# 精度を確認してみる
model.score(train_x, train_y)

0.546

このノートブックは以上です。
2-1-2のノートブックに続きます。